<a href="https://colab.research.google.com/github/Pavadareni/AI_Training/blob/main/Chatbot_Rag/Rag_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers faiss-cpu sentence-transformers wikipedia-api


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 74.1 MB/s eta 0:00:00
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.7.1-py3-none-any.whl size=14346 sha256=7bfafd752c12e2a9356aa98ac992708bb0dbbf89fc373a69400b15bbc2be520d
  Stored in directory: /root/.cache/pip/wheels/4c/96/18/b9201cc3e8b47b02b510460210cfd832ccf10c0c4dd0522962
Successfully built wikipedia-api


In [2]:
import faiss
from sentence_transformers import SentenceTransformer

# Load the Sentence Transformer for creating embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

# Load the knowledge base
with open("knowledge.txt", "r") as file:
    knowledge_base = file.read().splitlines()

# Create embeddings
kb_embeddings = model.encode(knowledge_base)

# Build FAISS index
index = faiss.IndexFlatL2(kb_embeddings.shape[1])  # L2 distance
index.add(kb_embeddings)


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [3]:
def retrieve_relevant_knowledge(query, k=2):
    """
    Retrieves the top-k relevant pieces of knowledge for a query.
    """
    query_embedding = model.encode([query])
    distances, indices = index.search(query_embedding, k)
    relevant_docs = [knowledge_base[idx] for idx in indices[0]]
    return "\n".join(relevant_docs)


In [4]:
from transformers import pipeline

# Load a pre-trained text generation model (e.g., T5 or GPT-2)
generator = pipeline("text2text-generation", model="t5-small")

def generate_response(context, query):
    """
    Generates a response based on the retrieved context and user query.
    """
    input_text = f"Context: {context} Query: {query}"
    response = generator(input_text, max_length=50, truncation=True)
    return response[0]['generated_text']


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [5]:
def chat_with_rag(query):
    """
    A pipeline combining retrieval and generation to respond to user queries.
    """
    # Step 1: Retrieve relevant knowledge
    context = retrieve_relevant_knowledge(query)

    # Step 2: Generate a response
    response = generate_response(context, query)
    return response


In [7]:
if __name__ == "__main__":
    print("RAG Chatbot: Ask me about colleges, courses, fees, or placements.")
    while True:
        user_query = input("You: ")
        if user_query.lower() in ["exit", "quit"]:
            print("RAG Chatbot: Goodbye!")
            break
        bot_response = chat_with_rag(user_query)
        print(f"RAG Chatbot: {bot_response}")


RAG Chatbot: Ask me about colleges, courses, fees, or placements.
You: What are the courses at IIT Delhi?
RAG Chatbot: : IIT Delhi offers courses like Civil Engineering, Chemical Engineering, and Computer Science. IIT Bombay offers courses like Computer Science, Mechanical Engineering, and Electrical Engineering.
You: Tell me about placements at IIT Bombay.
RAG Chatbot: : IIT Bombay offers courses like Computer Science, Mechanical Engineering, and Electrical Engineering. Context: IIT Bombay offers courses like Computer Science, Mechanical Engineering, and Electrical Engineering.
You: exit
RAG Chatbot: Goodbye!
